In [1]:
from pyspark.sql import SparkSession

In [2]:
spark1 = SparkSession.builder.appName("Test").getOrCreate()

22/12/20 20:39:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark1

In [4]:
spark1._sc

<SparkContext master=local[*] appName=Test>

In [5]:
def hscroll(activate=True):
    """activate/deactivate horizontal scrolling for wide output cells"""
    from IPython.display import display, HTML
    style = ('pre-wrap','pre')[activate] # select white-space style
    display(HTML("<style>pre {white-space: %s !important}</style>" % style))

In [6]:
# Read a csv file with headers using an input schema. The column names in the schema does not match with the
# given headers, so a warning will be generated

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, FloatType, DateType, TimestampType

fire_schema = StructType (
    [
        StructField('CallNumber', IntegerType(), True),
        StructField('UnitID', StringType(), True),
        StructField('IncidentNumber', IntegerType(), True),
        StructField('CallType', StringType(), True),
        StructField('CallDate', DateType(), True),
        StructField('WatchDate', DateType(), True),
        StructField('ReceivedDate', StringType(), True),
        StructField('EntryDate', TimestampType(), True),
        StructField('DispatchDate', TimestampType(), True),
        StructField('ResponseDate', TimestampType(), True),
        StructField('SceneDate', TimestampType(), True),
        StructField('TransportDate', TimestampType(), True),
        StructField('HospitalDate', TimestampType(), True),
        StructField('CallFinalDisposition', StringType(), True),
        StructField('AvailableDtTm', TimestampType(), True),
        StructField('Address', StringType(), True),       
        StructField('City', StringType(), True),       
        StructField('Zipcode', IntegerType(), True),       
        StructField('Battalion', StringType(), True),                 
        StructField('StationArea', StringType(), True),       
        StructField('Box', StringType(), True),       
        StructField('OriginalPriority', StringType(), True),       
        StructField('Priority', StringType(), True),       
        StructField('FinalPriority', IntegerType(), True),       
        StructField('ALSUnit', BooleanType(), True),       
        StructField('CallTypeGroup', StringType(), True),
        StructField('NumAlarms', IntegerType(), True),
        StructField('UnitType', StringType(), True),
        StructField('UnitSequenceInCallDispatch', IntegerType(), True),
        StructField('FirePreventionDistrict', StringType(), True),
        StructField('SupervisorDistrict', StringType(), True),
        StructField('NeighborhoodBoundaries', StringType(), True),
        StructField('RowID', StringType(), True),
        StructField('Location', StringType(), True),
        StructField('AnalysisNeighborhoods', FloatType(), True)
    ]
)

sf_fire_file = "data/Fire_Department_Calls_for_Service_100K.csv"

# fire_df = spark1.read..csv(sf_fire_file, header=True, schema=fire_schema)

fire_df = (spark1.read
            .format("csv")
            .option("header", "true")
            .schema(fire_schema)
            .option("enforceSchema", "true")
            .option("delimiter", ",")
            .option("dateFormat", "MM/dd/yyyy")
            .option("timestampFormat", "MM/dd/yyyy hh:mm:ss a")
            .load(sf_fire_file)
          )#.filter("callNumber is not null")


In [7]:
hscroll()
fire_df.show()

22/12/20 20:39:46 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/12/20 20:39:49 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Call Number, Unit ID, Incident Number, Call Type, Call Date, Watch Date, Received DtTm, Entry DtTm, Dispatch DtTm, Response DtTm, On Scene DtTm, Transport DtTm, Hospital DtTm, Call Final Disposition, Available DtTm, Address, City, Zipcode of Incident, Battalion, Station Area, Box, Original Priority, Priority, Final Priority, ALS Unit, Call Type Group, Number of Alarms, Unit Type, Unit sequence in call dispatch, Fire Prevention District, Supervisor District, Neighborhooods - Analysis Boundaries, RowID, case_location, Analysis Neighborhoods
 Schema: CallNumber, UnitID, IncidentNumber, CallType, CallDate, WatchDate, ReceivedDate, EntryDate, DispatchDate, ResponseDate, SceneDate, TransportDate, HospitalDate, CallFinalDisp

+----------+------+--------------+--------------------+----------+----------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+--------------------+-------------+-------+---------+-----------+----+----------------+--------+-------------+-------+--------------------+---------+--------+--------------------------+----------------------+------------------+----------------------+---------------+--------------------+---------------------+
|CallNumber|UnitID|IncidentNumber|            CallType|  CallDate| WatchDate|        ReceivedDate|          EntryDate|       DispatchDate|       ResponseDate|          SceneDate|      TransportDate|       HospitalDate|CallFinalDisposition|      AvailableDtTm|             Address|         City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|       CallTypeGroup|NumAlarms|UnitType|UnitSequenceIn

In [8]:
fire_df.count()

100000

In [9]:
# Translate a column from string to date
from pyspark.sql.functions import to_date, to_timestamp, col
fire_df = fire_df.withColumn("ReceivedDate", to_timestamp(col("ReceivedDate"),"MM/dd/yyyy hh:mm:ss a"))

In [10]:
fire_df.show()

+----------+------+--------------+--------------------+----------+----------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+--------------------+-------------+-------+---------+-----------+----+----------------+--------+-------------+-------+--------------------+---------+--------+--------------------------+----------------------+------------------+----------------------+---------------+--------------------+---------------------+
|CallNumber|UnitID|IncidentNumber|            CallType|  CallDate| WatchDate|       ReceivedDate|          EntryDate|       DispatchDate|       ResponseDate|          SceneDate|      TransportDate|       HospitalDate|CallFinalDisposition|      AvailableDtTm|             Address|         City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|       CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCa

22/12/20 20:40:24 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Call Number, Unit ID, Incident Number, Call Type, Call Date, Watch Date, Received DtTm, Entry DtTm, Dispatch DtTm, Response DtTm, On Scene DtTm, Transport DtTm, Hospital DtTm, Call Final Disposition, Available DtTm, Address, City, Zipcode of Incident, Battalion, Station Area, Box, Original Priority, Priority, Final Priority, ALS Unit, Call Type Group, Number of Alarms, Unit Type, Unit sequence in call dispatch, Fire Prevention District, Supervisor District, Neighborhooods - Analysis Boundaries, RowID, case_location, Analysis Neighborhoods
 Schema: CallNumber, UnitID, IncidentNumber, CallType, CallDate, WatchDate, ReceivedDate, EntryDate, DispatchDate, ResponseDate, SceneDate, TransportDate, HospitalDate, CallFinalDisposition, AvailableDtTm, Address, City, Zipcode, Battalion, StationArea, Box, OriginalPriority, Priority, FinalPriority, ALSUnit, CallTypeGroup, NumAlarms, UnitType, UnitSequenceInC

In [11]:
# Projections and Filters

few_fire_df = (
    fire_df
    .select("IncidentNumber", "AvailableDtTm", "CallType")
    .where(col("CallType") != "Medical Incident")
)

few_fire_df.show(5, truncate=False)
few_fire_df.count()

22/12/20 20:40:33 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Incident Number, Call Type, Available DtTm
 Schema: IncidentNumber, CallType, AvailableDtTm
Expected: IncidentNumber but found: Incident Number
CSV file: file:///opt/spark/notebooks/data/Fire_Department_Calls_for_Service_100K.csv


+--------------+-------------------+------------+
|IncidentNumber|AvailableDtTm      |CallType    |
+--------------+-------------------+------------+
|22054955      |2022-05-01 03:05:00|Outside Fire|
|22008871      |2022-01-19 02:35:26|Alarms      |
|21053032      |2021-05-03 21:38:09|Alarms      |
|21127914      |2021-10-20 22:25:52|Alarms      |
|22054815      |2022-04-30 18:40:08|Alarms      |
+--------------+-------------------+------------+
only showing top 5 rows



22/12/20 20:40:33 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Call Type
 Schema: CallType
Expected: CallType but found: Call Type
CSV file: file:///opt/spark/notebooks/data/Fire_Department_Calls_for_Service_100K.csv


32437

In [12]:
# Show the schema of a dataframe
fire_df.schema.json()

'{"fields":[{"metadata":{},"name":"CallNumber","nullable":true,"type":"integer"},{"metadata":{},"name":"UnitID","nullable":true,"type":"string"},{"metadata":{},"name":"IncidentNumber","nullable":true,"type":"integer"},{"metadata":{},"name":"CallType","nullable":true,"type":"string"},{"metadata":{},"name":"CallDate","nullable":true,"type":"date"},{"metadata":{},"name":"WatchDate","nullable":true,"type":"date"},{"metadata":{},"name":"ReceivedDate","nullable":true,"type":"timestamp"},{"metadata":{},"name":"EntryDate","nullable":true,"type":"timestamp"},{"metadata":{},"name":"DispatchDate","nullable":true,"type":"timestamp"},{"metadata":{},"name":"ResponseDate","nullable":true,"type":"timestamp"},{"metadata":{},"name":"SceneDate","nullable":true,"type":"timestamp"},{"metadata":{},"name":"TransportDate","nullable":true,"type":"timestamp"},{"metadata":{},"name":"HospitalDate","nullable":true,"type":"timestamp"},{"metadata":{},"name":"CallFinalDisposition","nullable":true,"type":"string"},{"m

In [13]:
# Distinct count aggregation

from pyspark.sql.functions import countDistinct

(
    fire_df
    .select("CallType")
    .where(col("CallType").isNotNull())
    .agg(countDistinct("CallType").alias("DistinctCallType"))
    .show()
)

22/12/20 20:40:53 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Call Type
 Schema: CallType
Expected: CallType but found: Call Type
CSV file: file:///opt/spark/notebooks/data/Fire_Department_Calls_for_Service_100K.csv


+----------------+
|DistinctCallType|
+----------------+
|              30|
+----------------+



In [14]:
# Just show distinct values, sorted

(
    fire_df
    .select("CallType")
    .where(col("CallType").isNotNull())
    .distinct()
    .sort("CallType")
    .show(100, truncate=False)
)

22/12/20 20:41:02 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Call Type
 Schema: CallType
Expected: CallType but found: Call Type
CSV file: file:///opt/spark/notebooks/data/Fire_Department_Calls_for_Service_100K.csv


+--------------------------------------------+
|CallType                                    |
+--------------------------------------------+
|Administrative                              |
|Alarms                                      |
|Assist Police                               |
|Citizen Assist / Service Call               |
|Confined Space / Structure Collapse         |
|Electrical Hazard                           |
|Elevator / Escalator Rescue                 |
|Explosion                                   |
|Extrication / Entrapped (Machinery, Vehicle)|
|Fuel Spill                                  |
|Gas Leak (Natural and LP Gases)             |
|HazMat                                      |
|High Angle Rescue                           |
|Industrial Accidents                        |
|Marine Fire                                 |
|Medical Incident                            |
|Mutual Aid / Assist Outside Agency          |
|Odor (Strange / Unknown)                    |
|Oil Spill   

In [15]:
## Use year function, rename column, perform distinct and sort

from pyspark.sql.functions import year

(
    fire_df
        .withColumnRenamed("CallDate", "IncidentDate")
        .select(year("IncidentDate").alias("IncidentYear"))
        .distinct()
        .orderBy("IncidentYear")
        .show()
)

22/12/20 20:41:11 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Call Date
 Schema: CallDate
Expected: CallDate but found: Call Date
CSV file: file:///opt/spark/notebooks/data/Fire_Department_Calls_for_Service_100K.csv


+------------+
|IncidentYear|
+------------+
|        2001|
|        2004|
|        2005|
|        2008|
|        2009|
|        2010|
|        2018|
|        2020|
|        2021|
|        2022|
+------------+



In [16]:
# Group By Count

(
    fire_df
    .select("CallType")
    .where(col("CallType").isNotNull())
    .groupBy("CallType")
    .count()
    .orderBy("count", asciiending=False)
    .show(n=10, truncate=False)
)

22/12/20 20:41:17 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Call Type
 Schema: CallType
Expected: CallType but found: Call Type
CSV file: file:///opt/spark/notebooks/data/Fire_Department_Calls_for_Service_100K.csv


+--------------------------------------------+-----+
|CallType                                    |count|
+--------------------------------------------+-----+
|Marine Fire                                 |1    |
|Train / Rail Incident                       |1    |
|Oil Spill                                   |2    |
|Mutual Aid / Assist Outside Agency          |3    |
|Suspicious Package                          |3    |
|Confined Space / Structure Collapse         |6    |
|Train / Rail Fire                           |7    |
|Administrative                              |7    |
|HazMat                                      |14   |
|Extrication / Entrapped (Machinery, Vehicle)|21   |
+--------------------------------------------+-----+
only showing top 10 rows



In [17]:
# Use aggregation functions

import pyspark.sql.functions as F
from pyspark.sql.functions import unix_timestamp

(
    fire_df
    .withColumn("ResponseDelayedMinutes", (unix_timestamp(col("DispatchDate")) - unix_timestamp(col("CallDate")))/60)
    .select(
        F.sum("NumAlarms"),
        F.avg("ResponseDelayedMinutes"),
        F.min("ResponseDelayedMinutes"),
        F.max("ResponseDelayedMinutes")
    )
    .show(n=100, truncate=False)
)

22/12/20 20:41:24 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Call Date, Dispatch DtTm, Number of Alarms
 Schema: CallDate, DispatchDate, NumAlarms
Expected: CallDate but found: Call Date
CSV file: file:///opt/spark/notebooks/data/Fire_Department_Calls_for_Service_100K.csv


+--------------+---------------------------+---------------------------+---------------------------+
|sum(NumAlarms)|avg(ResponseDelayedMinutes)|min(ResponseDelayedMinutes)|max(ResponseDelayedMinutes)|
+--------------+---------------------------+---------------------------+---------------------------+
|100269        |807.1473806666676          |0.1                        |1525.7666666666667         |
+--------------+---------------------------+---------------------------+---------------------------+



In [18]:
# Use aggregation functions with a groupBy

import pyspark.sql.functions as F
from pyspark.sql.functions import unix_timestamp

(
    fire_df
    .withColumn("ResponseDelayedMinutes", (unix_timestamp(col("DispatchDate")) - unix_timestamp(col("CallDate")))/60)
    .groupBy("CallType")
    .agg(
        F.sum("NumAlarms"),
        F.avg("ResponseDelayedMinutes"),
        F.min("ResponseDelayedMinutes"),
        F.max("ResponseDelayedMinutes")
    )
    .show(n=100, truncate=False)
)

22/12/20 20:41:29 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Call Type, Call Date, Dispatch DtTm, Number of Alarms
 Schema: CallType, CallDate, DispatchDate, NumAlarms
Expected: CallType but found: Call Type
CSV file: file:///opt/spark/notebooks/data/Fire_Department_Calls_for_Service_100K.csv


+--------------------------------------------+--------------+---------------------------+---------------------------+---------------------------+
|CallType                                    |sum(NumAlarms)|avg(ResponseDelayedMinutes)|min(ResponseDelayedMinutes)|max(ResponseDelayedMinutes)|
+--------------------------------------------+--------------+---------------------------+---------------------------+---------------------------+
|Elevator / Escalator Rescue                 |269           |869.4080545229249          |7.033333333333333          |1433.6                     |
|Marine Fire                                 |1             |1167.5833333333333         |1167.5833333333333         |1167.5833333333333         |
|Confined Space / Structure Collapse         |6             |1263.95                    |1263.95                    |1263.95                    |
|Administrative                              |7             |834.5095238095239          |210.68333333333334         |1113.4 

In [20]:
# Use Row

from pyspark.sql import Row

authors_schema = StructType([
	StructField("Author" , StringType(), False), 
	StructField("State", StringType(), False)])

rows = [Row("Matei Zaharia", "CA"), Row("Reynold Xin", "CA")]
authors_df = spark1.createDataFrame(rows, authors_schema)
authors_df.show()


+-------------+-----+
|       Author|State|
+-------------+-----+
|Matei Zaharia|   CA|
|  Reynold Xin|   CA|
+-------------+-----+



In [23]:
## Using expr, Array of Strings

from pyspark.sql.types import ArrayType

blogs_schema = StructType([
   StructField("Id", IntegerType(), False),
   StructField("First", StringType(), False),
   StructField("Last", StringType(), False),
   StructField("Url", StringType(), False),
   StructField("Published", StringType(), False),
   StructField("Hits", IntegerType(), False),
   StructField("Campaigns", ArrayType(StringType()), False)])

#create our data
blogs_data = [[1, "Jules", "Damji", "https://tinyurl.1", "1/4/2016", 4535, ["twitter", "LinkedIn"]],
       [2, "Brooke","Wenig","https://tinyurl.2", "5/5/2018", 8908, ["twitter", "LinkedIn"]],
       [3, "Denny", "Lee", "https://tinyurl.3","6/7/2019",7659, ["web", "twitter", "FB", "LinkedIn"]],
       [4, "Tathagata", "Das","https://tinyurl.4", "5/12/2018", 10568, ["twitter", "FB"]],
       [5, "Matei","Zaharia", "https://tinyurl.5", "5/14/2014", 40578, ["web", "twitter", "FB", "LinkedIn"]],
       [6, "Reynold", "Xin", "https://tinyurl.6", "3/2/2015", 25568, ["twitter", "LinkedIn"]]
      ]

blogs_df = spark1.createDataFrame(blogs_data, blogs_schema)
blogs_df.show()

+---+---------+-------+-----------------+---------+-----+--------------------+
| Id|    First|   Last|              Url|Published| Hits|           Campaigns|
+---+---------+-------+-----------------+---------+-----+--------------------+
|  1|    Jules|  Damji|https://tinyurl.1| 1/4/2016| 4535| [twitter, LinkedIn]|
|  2|   Brooke|  Wenig|https://tinyurl.2| 5/5/2018| 8908| [twitter, LinkedIn]|
|  3|    Denny|    Lee|https://tinyurl.3| 6/7/2019| 7659|[web, twitter, FB...|
|  4|Tathagata|    Das|https://tinyurl.4|5/12/2018|10568|       [twitter, FB]|
|  5|    Matei|Zaharia|https://tinyurl.5|5/14/2014|40578|[web, twitter, FB...|
|  6|  Reynold|    Xin|https://tinyurl.6| 3/2/2015|25568| [twitter, LinkedIn]|
+---+---------+-------+-----------------+---------+-----+--------------------+



In [27]:
from pyspark.sql.functions import expr

blogs_df.select((expr("Hits") * 2).alias("HitsX2")).show(2)
blogs_df.select((col("Hits") * 2).alias("HitsX2")).show(2)
blogs_df.select(expr("Hits * 2").alias("HitsX2")).show(2)

# show heavy hitters
blogs_df.withColumn("Big Hitters", (expr("Hits > 10000"))).show()

print(blogs_df.schema)


+------+
|HitsX2|
+------+
|  9070|
| 17816|
+------+
only showing top 2 rows

+------+
|HitsX2|
+------+
|  9070|
| 17816|
+------+
only showing top 2 rows

+------+
|HitsX2|
+------+
|  9070|
| 17816|
+------+
only showing top 2 rows

+---+---------+-------+-----------------+---------+-----+--------------------+-----------+
| Id|    First|   Last|              Url|Published| Hits|           Campaigns|Big Hitters|
+---+---------+-------+-----------------+---------+-----+--------------------+-----------+
|  1|    Jules|  Damji|https://tinyurl.1| 1/4/2016| 4535| [twitter, LinkedIn]|      false|
|  2|   Brooke|  Wenig|https://tinyurl.2| 5/5/2018| 8908| [twitter, LinkedIn]|      false|
|  3|    Denny|    Lee|https://tinyurl.3| 6/7/2019| 7659|[web, twitter, FB...|      false|
|  4|Tathagata|    Das|https://tinyurl.4|5/12/2018|10568|       [twitter, FB]|       true|
|  5|    Matei|Zaharia|https://tinyurl.5|5/14/2014|40578|[web, twitter, FB...|       true|
|  6|  Reynold|    Xin|https://tiny

In [31]:
departuredelays_csv_file = "data/departuredelays.csv"

departuredelays_schema = "`date` STRING, `delay` INT, `distance` INT, `origin` STRING, `destination` STRING"

df = (
    spark1.read
    .format("csv")
    .schema(departuredelays_schema)
    .option("enforceSchema", "true")
    # .option("inferSchema", "true")
    .option("header", "true")
    .load(departuredelays_csv_file)
)
df.createOrReplaceTempView("us_delay_flights")

In [34]:
spark1.sql(
"""
select distance, origin, destination
from us_delay_flights
where distance > 1000
order by distance desc
limit 10
"""
).show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+



In [35]:
spark1.sql("""
select date, delay, origin, destination
from us_delay_flights
where delay > 120
and origin = 'SFO'
and destination = 'ORD'
order by delay desc
""").show(10)

+--------+-----+------+-----------+
|    date|delay|origin|destination|
+--------+-----+------+-----------+
|02190925| 1638|   SFO|        ORD|
|01031755|  396|   SFO|        ORD|
|01022330|  326|   SFO|        ORD|
|01051205|  320|   SFO|        ORD|
|01190925|  297|   SFO|        ORD|
|02171115|  296|   SFO|        ORD|
|01071040|  279|   SFO|        ORD|
|01051550|  274|   SFO|        ORD|
|03120730|  266|   SFO|        ORD|
|01261104|  258|   SFO|        ORD|
+--------+-----+------+-----------+
only showing top 10 rows



In [46]:
spark1.sql("""
select delay, origin, destination,
       case
         when delay > 360 then 'Very Long Delays'
         when delay >= 120 and delay <= 360 then 'Long Delays'
         when delay >= 60 and delay < 120 then 'Short Delays'
         when delay > 0 and delay < 60 then 'Tolerable Delays'
         when delay = 0 then 'No Delay'
         else 'Early'
       end as flight_delays
from us_delay_flights
order by origin, delay desc
""").show(10)

+-----+------+-----------+-------------+
|delay|origin|destination|flight_delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|  Long Delays|
|  305|   ABE|        ATL|  Long Delays|
|  275|   ABE|        ATL|  Long Delays|
|  257|   ABE|        ATL|  Long Delays|
|  247|   ABE|        ATL|  Long Delays|
|  247|   ABE|        DTW|  Long Delays|
|  219|   ABE|        ORD|  Long Delays|
|  211|   ABE|        ATL|  Long Delays|
|  197|   ABE|        DTW|  Long Delays|
|  192|   ABE|        ORD|  Long Delays|
+-----+------+-----------+-------------+
only showing top 10 rows



In [52]:
spark1.catalog.listTables()
# spark1.catalog.listColumns("us_delay_flights", dbName=None)
help(spark1.catalog.listColumns)

Help on method listColumns in module pyspark.sql.catalog:

listColumns(tableName, dbName=None) method of pyspark.sql.catalog.Catalog instance
    Returns a list of columns for the given table/view in the specified database.
    
    If no database is specified, the current database is used.
    
    Note: the order of arguments here is different from that of its JVM counterpart
    because Python does not support method overloading.
    
    .. versionadded:: 2.0

